In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

## Classes & Functions

#### Algorithms

In [ ]:
class SimulatedAnnealing:
    def __init__(self, current, temperature, nodes, steps):
        self.initial_solution = current
        self.current_solution = current
        self.temperature = temperature
        self.steps = steps
        self.nodes = nodes
        self.n = len(self.initial_solution)
        self.rate = 0.99

        self.current_score = self.distance(self.initial_solution)

    def distance(self, solution) -> int:
        distance = 0
        for i in range(1, self.n):
            distance += self.nodes[solution[i] - 1].distance(self.nodes[solution[i - 1] - 1])
        return distance

    def update_temperature(self) -> None:
        self.temperature = self.temperature * self.rate

    def swap(self, solution) -> list:
        r_1, r_2, r_3, r_4 = np.random.randint(0, self.n, 4)
        solution[r_1], solution[r_2], solution[r_3], solution[r_4] = solution[r_4], solution[r_3], solution[r_2], solution[r_1]
        return solution

    def run(self) -> list:
        for _ in range(self.steps):
            cur_sol = self.current_solution
            new_solution = self.swap(cur_sol)
            new_score = self.distance(new_solution)
            print(new_score)
            print(f'the current{self.current_score}')

            if self.current_score > new_score:
                self.current_score = new_score
                self.current_solution = new_solution
            else:
                score_diff = self.current_score - new_score
                if np.exp(score_diff/self.temperature) > random.random():
                    self.current_solution = new_solution
                    self.current_score = new_score

            self.update_temperature()

        return self.current_solution, self.current_score

In [ ]:
class Greedy:
    def __init__(self, nodes):
        self.network = nodes
        self.current_city = self.starting_point()
        self.n = len(self.network)
        self.order = [self.current_city]
        self.distance = 0

    def starting_point(self, number=1) -> int:
        return number

    def next_city(self) -> None:
        current_node = self.network[self.current_city - 1]
        connections = current_node.get_connections()

        sorted_connections = {k: v for k, v in sorted(connections.items(),
            key=lambda item: item[1])}

        for con in sorted_connections:
            if con not in self.order:
                self.order.append(con)
                self.distance += current_node.distance(self.network[con - 1])
                self.current_city = con
                break

    def get_solution(self):
        return self.order

    def get_distance(self):
        return self.distance

    def run(self) -> list:
        while len(self.order) < self.n:
            self.next_city()
        return self.order, self.distance

In [ ]:
class Random:
    def __init__(self, nodes):
        pass

#### Network

In [ ]:
class Node:
    def __init__(self, x, y, number):
        self.number = number
        self.x = x
        self.y = y
        self.connections = {}

    def distance(self, b) -> int:
        return np.linalg.norm(np.array(self.get_coordinates())
            - np.array(b.get_coordinates()))

    def get_coordinates(self) -> list:
        return [self.x, self.y]

    def create_connections(self, nodes) -> None:
        for node in nodes:
            if node.number == self.number:
                continue
            self.connections[node.number] = self.distance(node)

    def get_connections(self) -> dict:
        return self.connections

#### Helper

In [ ]:
def load_data(file_name) -> list:
    """
    Loads in a text file with the given file name and creates a node for every
    point. Returns a list of nodes.
    """
    contents = open(f'TSP-Configurations/{file_name}.txt', "r")
    contents = contents.read().splitlines()[6:-1]
    nodes = list()

    # Go through every line of the file and create node for every line.
    for node in contents:
        values = [int(value) for value in node.split()]
        nodes.append(Node(values[1], values[2], values[0]))

    # Return the list of nodes.
    return nodes


In [ ]:
def create_connections(nodes) -> list:
    """
    Function that connections a city with every other city.
    Returns a list with connected nodes.
    """
    for node in nodes:
        node.create_connections(nodes)
    return nodes

In [ ]:
def distance_travelled(solution, nodes) -> int:
    distance = 0
    for i in range(1, len(solution)):
        distance += nodes[solution[i]].distance(nodes[solution[i - 1]])
    return distance

In [ ]:
def calculate_optimal():
    pass

## Testing

Testing happens on the configuration of the eli51.tsp file. For experiments the bigger
network a280.tsp is used. 

In [ ]:
file_name = 'eli51.tsp'

In [ ]:
# Load in data and create the connections.
data = load_data(file_name)
nodes = create_connections(data)

# Create Greedy object and run the algorithm to get a base network.
greedy = Greedy(nodes)
results_greedy, distance_1 = greedy.run()
siman = SimulatedAnnealing(results_greedy, 100, nodes)
print(distance_1)
print(siman.run())

## Experiments